## Steps to Replace Certs

1. Download certs from the GoDaddy account
2. Unzip the certs. It will create to folders, one for RoadmapPartners and one for RetirementTaxAnalyzer.
3. SCP or RSync those files to the server
4. Move the files into `/etc/ssl/certs`
5. the `gd_bundle-g2-g1.crt` is the same in both folders and you can use either one.
6. `cp` the `.crt` files from the respective domains onto and overwriting the existing `retirementtaxanalyzer.crt` and `roadmappartners.net.crt`.
   1. You will have to keep track of which file came from which domain folder to overwrite the right cert.
7. `sudo systemctl stop tomcat`
8. `sudo systemctl start tomcat`
9.  Run the `curl-cert-check.sh` in WSL/linux
10. Automate this process in Ansible or something better.

In [ ]:
$uat = 'ubuntu@54.177.161.104'
$prod = 'ubuntu@184.72.30.45'
# May have to start UAT with `Start-PrsUat`
echo "$($uat)"

$target = $uat

In [ ]:
$roadmapCert = "2f570f84a8b1836b"
$rtaCert = "b9c38194ee3c4be7"

scp "_.roadmappartners.net/$roadmapCert.crt" "$($target):~/"
ssh "$target" "sudo mv ~/$roadmapCert.crt /etc/ssl/certs/roadmappartners.net.crt"

scp ".\retirementtaxanalyzer.com\$rtaCert.crt" "$($target):~/"
ssh "$target" "sudo mv ~/$rtaCert.crt /etc/ssl/certs/retirementtaxanalyzer.crt"

scp ".\retirementtaxanalyzer.com\gd_bundle-g2-g1.crt" "$($target):~/"
ssh $target "sudo cp ~/gd_bundle-g2-g1.crt /etc/ssl/certs/gd_bundle-g2-g1.crt"


In [ ]:
# ssh $target "sudo systemctl stop tomcat" # Might not be required. Apache2 restart is required.
# ssh $target "sudo systemctl start tomcat"
ssh $target "sudo systemctl stop apache2"
ssh $target "sudo systemctl start apache2"

ssh $target "ls /etc/ssl/certs/*.crt -la"

In [2]:
. $PrsPath
Start-PrsUat


InvalidOperation: The expression after '.' in a pipeline element produced an object that was not valid. It must result in a command name, a script block, or a CommandInfo object.
Start-PrsUat: 
Line |
   2 |  Start-PrsUat
     |  ~~~~~~~~~~~~
     | The term 'Start-PrsUat' is not recognized as a name of a cmdlet, function, script file, or executable program.
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.


Error: Command failed: SubmitCode: . $PrsPath ...

In [ ]:
## May have to run on the command line
Show-CertificateExpirationDate

In [ ]:
curl --insecure -vvI 'https://roadmappartners.net' 2>&1 

In [ ]:
curl --insecure -vvI 'https://uat.roadmappartners.net' 2>&1 | awk 'BEGIN { cert=0 } /^\* Server certificate:/ { cert=1 } /^\*/ { if (cert) print }'

In [ ]:
openssl s_client -connect icanhazip.com:443 -showcerts

In [ ]:
# This one works better but you might have to copy it out to the WSL terminal
$domain="uat.retirementtaxanalyzer.com"
echo | openssl s_client -showcerts -servername $domain -connect gnupg.org:443 2>/dev/null | openssl x509 -inform pem -noout -text >> "$domain.openssl-showcerts.yml"
